In [1]:
%matplotlib inline
import numpy as np
import time
import h5py
import keras
import pandas as pd
import math
import joblib
import os, cv2
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import display

from keras.layers import (Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Activation, 
                          Dropout, Conv2D, Conv2DTranspose, MaxPooling2D)
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.initializers import RandomUniform
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Model
from keras import metrics
from keras.utils import np_utils
from keras import backend as K
from keras.datasets import mnist
import tensorflow as tf
from tensorflow import Graph, Session
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


# Variational Autoencoder Parameters

In [2]:
img_rows, img_cols, img_chns = 20, 20, 3

if K.image_data_format() == 'channels_first':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)

batch_size = 600
latent_dim = 128
intermediate_dim = 512
epsilon_std = 1.0
epochs = 100
activation = 'relu'
dropout = 0.5
learning_rate = 1e-4
decay = 0.0
num_classes = 2

image_name = '322.tif'
mask_name = '322_car_1'
data_path = './data/xview/'
unlabeled_imgs_path = './temp'
pos_folder_path = os.path.join(data_path, mask_name+'_samples', 'positive')
neg_folder_path = os.path.join(data_path, mask_name+'_samples', 'negative')

# Load map dataset

In [3]:
def read_img_from_dir(path):
    data= []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            img_path = os.path.join(root, name)
            data.append(cv2.imread(img_path))
    data = np.array(data)
    return data


In [4]:
X_others = read_img_from_dir(unlabeled_imgs_path)
pos_samples = read_img_from_dir(pos_folder_path)
neg_samples = read_img_from_dir(neg_folder_path)
print(X_others.shape, pos_samples.shape, neg_samples.shape)

(1406, 20, 20, 3) (5, 20, 20, 3) (5, 20, 20, 3)


In [5]:
X_others = X_others.reshape(X_others.shape[0], img_rows, img_cols, img_chns) / 255.
pos_samples = list(pos_samples.reshape(pos_samples.shape[0], img_rows, img_cols, img_chns) / 255.)
neg_samples = list(neg_samples.reshape(neg_samples.shape[0], img_rows, img_cols, img_chns) / 255.)
X = []
y = []
for i in range(len(pos_samples)):
    X.append(pos_samples[i])
    X.append(neg_samples[i])
    y.append([0,1])
    y.append([1,0])
X, y = np.array(X), np.array(y)
X_others = X_others[:batch_size*2]
print(X_others.shape, X.shape, y.shape)

(1200, 20, 20, 3) (10, 20, 20, 3) (10, 2)


# Encoder Network

In [6]:
def create_enc_conv_layers(stage, **kwargs):
    conv_name = '_'.join(['enc_conv', str(stage)])
    layers = [
        Conv2D(name=conv_name, **kwargs),
        Activation(activation),
    ]
    return layers

def create_dense_layers(stage, width):
    dense_name = '_'.join(['dense', str(stage)])
    layers = [
        Dense(width, name=dense_name),
        Activation(activation),
        Dropout(dropout),
    ]
    return layers

def inst_layers(layers, in_layer):
    x = in_layer
    for layer in layers:
        if isinstance(layer, list):
            x = inst_layers(layer, x)
        else:
            x = layer(x)
        
    return x

In [7]:

enc_filters=64
enc_layers = [
    create_enc_conv_layers(stage=1, filters=enc_filters, kernel_size=3, strides=1, padding='same'),
    create_enc_conv_layers(stage=2, filters=enc_filters, kernel_size=3, strides=1, padding='same'),
    create_enc_conv_layers(stage=3, filters=enc_filters, kernel_size=3, strides=2, padding='same'),
    Flatten(),
    create_dense_layers(stage=4, width=intermediate_dim),
]

In [8]:
# Labeled encoder

x_in = Input(shape=(img_rows, img_cols, img_chns))
y_in = Input(shape=(num_classes,))
_enc_dense = inst_layers(enc_layers, x_in)

_z_mean_1 = Dense(latent_dim)(_enc_dense)
_z_log_var_1 = Dense(latent_dim)(_enc_dense)

z_mean = _z_mean_1
z_log_var = _z_log_var_1


### Reparameterization Trick

In [9]:

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Classifier Network 

In [10]:
classifier_layers = [
    Conv2D(32, (3, 3), padding='same'),
    Activation('relu'),
    Conv2D(32, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512,name='dense_cls_'),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes,name='dense_cls'),
    Activation(tf.nn.softmax),
]

In [11]:
_cls_output = inst_layers(classifier_layers, x_in)
_y_output = _cls_output

# Decoder Network

In [12]:
def create_dec_trans_conv_layers(stage, **kwargs):
    conv_name = '_'.join(['dec_trans_conv', str(stage)])
    layers = [
        Conv2DTranspose(name=conv_name, **kwargs),
        Activation(activation),
    ]
    return layers

In [13]:


dec_filters = 64

decoder_layers = [
    create_dense_layers(stage=10, width=10 * 10 * 64),
    Reshape((10, 10, 64)),
    create_dec_trans_conv_layers(11, filters=dec_filters, kernel_size=3, strides=1, padding='same'),
    create_dec_trans_conv_layers(12, filters=dec_filters, kernel_size=3, strides=1, padding='same'),
    create_dec_trans_conv_layers(13, filters=dec_filters, kernel_size=3, strides=2, padding='same'),
    Conv2DTranspose(name='x_decoded', filters=img_chns, kernel_size=1, strides=1, activation='sigmoid'),
]

In [14]:

# Labeled decoder
_merged = concatenate([y_in, z])

_dec_out = inst_layers(decoder_layers, _merged)
_x_output = _dec_out



In [15]:

# Unlabeled decoder
u_merged = concatenate([_y_output, z])
u_dec_out = inst_layers(decoder_layers, u_merged)
u_x_output = u_dec_out

# Loss Function

In [16]:
def kl_loss(x, x_decoded_mean, z_mean=z_mean, z_log_var=z_log_var):
    kl_loss = - 0.5 * K.sum(1. + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
   
    return K.mean(kl_loss)

def logxy_loss(x, x_decoded_mean):
    x = K.flatten(x)
    x_decoded_mean = K.flatten(x_decoded_mean)
    xent_loss = img_rows * img_cols * img_chns * metrics.binary_crossentropy(x, x_decoded_mean)
   
    # p(y) for observed data is equally distributed
    logy = np.log(1. / num_classes)
    
    return xent_loss - logy

def labeled_vae_loss(x, x_decoded_mean):
    return logxy_loss(x, x_decoded_mean) + kl_loss(x, x_decoded_mean)

def cls_loss(y, y_pred, N=1000):
    alpha = 0.1 * N
    return alpha * metrics.categorical_crossentropy(y, y_pred)

def unlabeled_vae_loss(x, x_decoded_mean):
    entropy = metrics.categorical_crossentropy(_y_output, _y_output)
    # This is probably not correct, see discussion here: https://github.com/bjlkeng/sandbox/issues/3
    labeled_loss = logxy_loss(x, x_decoded_mean) + kl_loss(x, x_decoded_mean)
    
    return K.mean(K.sum(_y_output * labeled_loss, axis=-1)) + entropy

# Compile Model

In [17]:
label_vae = Model(inputs=[x_in, y_in], outputs=[_x_output, _y_output])
# optimizer = Adam(lr=learning_rate, decay=decay)
optimizer = SGD(lr=learning_rate)
label_vae.compile(optimizer=optimizer, loss=[labeled_vae_loss, cls_loss])
label_vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20, 20, 3)    0                                            
__________________________________________________________________________________________________
enc_conv_1 (Conv2D)             (None, 20, 20, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 20, 20, 64)   0           enc_conv_1[0][0]                 
__________________________________________________________________________________________________
enc_conv_2 (Conv2D)             (None, 20, 20, 64)   36928       activation_1[0][0]               
__________________________________________________________________________________________________
activation

In [18]:
unlabeled_vae = Model(inputs=x_in, outputs=u_x_output)
# optimizer = Adam(lr=learning_rate, decay=decay)
optimizer = SGD(lr=learning_rate)
unlabeled_vae.compile(optimizer=optimizer, loss=unlabeled_vae_loss)
unlabeled_vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20, 20, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 20, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 20, 20, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 18, 18, 32)   9248        activation_5[0][0]               
__________________________________________________________________________________________________
activation

In [19]:

def fit_model(X_unlabeled, X_labeled, y_labeled, epochs):
    start = time.time()
    history = []
    
    for epoch in range(epochs):
        unlabeled_index = np.arange(len(X_unlabeled))
        np.random.shuffle(unlabeled_index)
        
        # Repeat the labeled data to match length of unlabeled data

        l = np.arange(len(X_labeled))
        np.random.shuffle(l)
        labeled_index = l
        
        batches = len(X_unlabeled) // batch_size
        
        for i in range(batches):
            # Labeled
            index_range =  labeled_index[i * batch_size:(i+1) * batch_size]
            
            loss = label_vae.train_on_batch([X_labeled[labeled_index], y_labeled[labeled_index]], 
                                            [X_labeled[labeled_index], y_labeled[labeled_index]])


            # Unlabeled
            index_range =  unlabeled_index[i * batch_size:(i+1) * batch_size]
            loss += [unlabeled_vae.train_on_batch(X_unlabeled[index_range],  X_unlabeled[index_range])]
            print('epoch '+str(epoch)+'   loss = ' +str(loss))
            history.append(loss)
                

    
   
    done = time.time()
    elapsed = done - start
    print("Elapsed: ", elapsed)
    
    return history

In [20]:
sample_size = 10
start = time.time()
print('Fitting with sample_size: {}'.format(sample_size))

history = fit_model(X_others, X, y, epochs=epochs)



Fitting with sample_size: 10
epoch 0   loss = [897.53796, 829.35254, 68.185394, 831.3213]
epoch 0   loss = [892.5414, 822.78516, 69.756195, 829.5373]
epoch 1   loss = [886.1562, 816.7145, 69.44171, 827.87]
epoch 1   loss = [880.24963, 810.673, 69.576645, 827.56726]
epoch 2   loss = [873.8706, 805.0731, 68.7975, 826.2311]
epoch 2   loss = [870.66174, 799.52405, 71.137665, 824.6471]
epoch 3   loss = [862.8779, 794.4896, 68.388275, 824.0182]
epoch 3   loss = [857.844, 789.1426, 68.7014, 823.5901]
epoch 4   loss = [852.43726, 784.2622, 68.175026, 823.47906]
epoch 4   loss = [849.39685, 779.42285, 69.97399, 821.7064]
epoch 5   loss = [843.67145, 775.21204, 68.459404, 820.9852]
epoch 5   loss = [840.2226, 770.5342, 69.6884, 822.85626]
epoch 6   loss = [835.0784, 766.5233, 68.555115, 822.27]
epoch 6   loss = [831.46436, 762.379, 69.085304, 820.78076]
epoch 7   loss = [825.60736, 758.4072, 67.20014, 821.5097]
epoch 7   loss = [824.82007, 755.22327, 69.59683, 821.6362]
epoch 8   loss = [819.568

epoch 68   loss = [736.5045, 688.66614, 47.83835, 843.38794]
epoch 68   loss = [728.3835, 689.09485, 39.288612, 846.59973]
epoch 69   loss = [733.8062, 688.94556, 44.860657, 845.0332]
epoch 69   loss = [734.54626, 689.2057, 45.340557, 844.9385]
epoch 70   loss = [732.65717, 689.0006, 43.65654, 843.8828]
epoch 70   loss = [732.3957, 689.1426, 43.25313, 846.2783]
epoch 71   loss = [735.87103, 689.0759, 46.795113, 845.11755]
epoch 71   loss = [731.6221, 688.6156, 43.006493, 844.8513]
epoch 72   loss = [731.673, 688.9503, 42.722626, 845.1708]
epoch 72   loss = [730.51843, 688.86304, 41.65542, 844.90826]
epoch 73   loss = [730.15204, 688.73254, 41.41951, 846.07263]
epoch 73   loss = [726.10944, 689.13257, 36.97684, 844.0471]
epoch 74   loss = [727.7316, 688.7661, 38.965515, 843.1204]
epoch 74   loss = [729.61487, 688.0608, 41.554077, 847.04]
epoch 75   loss = [727.22253, 689.0608, 38.161766, 843.4336]
epoch 75   loss = [733.0252, 688.83813, 44.1871, 846.62115]
epoch 76   loss = [727.9349, 6

## load test images

In [21]:
# load test images with index
stride = 10
X_test = []
X_test_idx = []
image = cv2.imread(os.path.join(data_path, image_name))
mask = cv2.imread(os.path.join(data_path, mask_name+'_label.jpeg'), 0) / 255
print(image.shape, mask.shape)

for row in range(0,mask.shape[0]-img_rows,stride):
    for col in range(0,mask.shape[1]-img_rows,stride):
        sub_mask = mask[row:row+img_rows, col:col+img_rows]
        if np.count_nonzero(sub_mask) == img_rows*img_rows:
            sub_img = image[row:row+img_rows, col:col+img_rows]
            X_test.append(sub_img)
            X_test_idx.append([row,col])
            
X_test = np.array(X_test) / 255.0
print(X_test.shape, len(X_test_idx))

(3063, 4764, 3) (3063, 4764)
(3666, 20, 20, 3) 3666


In [22]:
classifier = Model(inputs=[x_in], outputs=[_y_output])

y_pred = np.argmax(classifier.predict(X_test), axis=-1)

print(y_pred.shape)

(3666,)


In [23]:
pred_pos_idx, pred_neg_idx = [], []

for i in range(y_pred.shape[0]):
    if y_pred[i] == 0:
        pred_neg_idx.append(X_test_idx[i])
    else:
        pred_pos_idx.append(X_test_idx[i])
        
print(len(pred_pos_idx), len(pred_neg_idx))

744 2922


## generate point-level prediction results

In [24]:
# save the positive point-level prediction resuls as a txt file
save_file_name = mask_name+'_points_pred.txt'
save_file_path = os.path.join(data_path, save_file_name)
pred_center_points = []
for i in pred_pos_idx:
    x, y = i[0], i[1]
    x_c, y_c = int(x+img_rows/2), int(y+img_rows/2)
    pred_center_points.append([x_c,y_c])

pred_center_points = np.array(pred_center_points)
np.savetxt(save_file_path, pred_center_points, fmt='%d', delimiter=',')
print('save the point-level prediction results in ',save_file_path)

save the point-level prediction results in %s ./data/xview/322_car_1_points_pred.txt


## generate patch-level prediction map

In [25]:
pred_map = np.zeros(mask.shape)

for i in pred_pos_idx:
    x, y = i[0], i[1]
    pred_map[x:x+img_rows, y:y+img_rows] = 255

pred_path = os.path.join(data_path, mask_name+'_pred.jpeg')
cv2.imwrite(pred_path, pred_map)
print('save the patch-level prediction results in ',pred_path)

True

## generate point-level prediction map for visualization

In [26]:
import copy

radius = 1
# Blue color in BGR
color = (0, 0, 255)
# Line thickness of 2 px
thickness=2

pred_points_map = copy.deepcopy(image)
for i in pred_pos_idx:
    x, y = i[0], i[1]
    x_c, y_c = int(x+img_rows/2), int(y+img_rows/2)
    pred_points_map = cv2.circle(pred_points_map, (y_c,x_c), radius, color, thickness)

pred_points_path = os.path.join(data_path, mask_name+'_pred_points.jpeg')
cv2.imwrite(pred_points_path, pred_points_map)

True